In [23]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tdqm import tdqm
# 讀取數據
train_normal = np.load('Data/train_normal.npy')
train_outer_break = np.load('Data/train_outer_break.npy')
train_inner_break = np.load('Data/train_inner_break.npy')

# 創建標籤
labels_normal = np.zeros(len(train_normal))
labels_outer_break = np.ones(len(train_outer_break))
labels_inner_break = np.full(len(train_inner_break), 2)

# 合併數據和標籤
X_train = np.concatenate((train_normal, train_outer_break, train_inner_break), axis=0)
y_train = np.concatenate((labels_normal, labels_outer_break, labels_inner_break), axis=0)
print(X_train.shape)
print(y_train.shape)

X_train_tensor = torch.tensor(X_train, dtype=torch.float).unsqueeze(1)  # Adding channel dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Assuming classification task

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
batch_size = 1  # Adjust the batch size according to your needs and hardware capabilities

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



(750, 10240)
(750,)


In [24]:
class VibrationCNN(nn.Module):
    def __init__(self):
        super(VibrationCNN, self).__init__()
        # Assuming 1D Convolutional layers for sequence data
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # The size here might need adjustment based on the size after convolutions and pooling
        # Calculating the size: As input is 10240, after 3 pooling layers, the size is 10240/(2^3) = 1280
        self.fc1 = nn.Linear(128 * 1280, 512)
        
        # Final output layer for classification into classes
        # Assuming your output is multi-class classification with C classes
        self.fc2 = nn.Linear(512, 3)  # Replace C with the actual number of classes

    def forward(self, x):
        # x shape is [batch_size, channels, sequence_length]
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        # Adjust the number of pool operations according to your model architecture
        # An additional pool operation as an example
        x = self.pool(x)
        
        # Flatten for the fully connected layer
        x = x.view(-1, 128 * 1280)  # Adjust the flattening size accordingly
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation, as nn.CrossEntropyLoss() includes softmax
        
        return x


In [27]:
import torch.optim as optim
import base_func 
model_list = {
    "VibrationCNN" : lambda: VibrationCNN(),
}
model_name = [
    "VibrationCNN",
]
loaders = {
    'train': train_loader,
}
base_func.train(model_list, model_name, loaders)
# model = VibrationCNN().cuda()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# 假設你已經將X_train, y_train轉換成適當的PyTorch tensors和DataLoader

# for epoch in range(10):  # 迭代次數
#     for inputs, labels in tdqm(train_loader):
#         inputs, labels = inputs.cuda(), labels.cuda()
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print(outputs)
#         # print(labels)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         # break
#     print(f'Epoch {epoch+1}, Loss: {loss.item()}')


TypeError: unhashable type: 'list'